In [3]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import cross_val_score
import matplotlib
import seaborn as sb
import statsmodels.api as sm
import statsmodels.formula.api as smf


def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

%matplotlib inline

table_split = {'sales_prd': ['Semana','Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
              'sales_rtn':['Semana','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima', 'Demanda_uni_equil']}


store = pd.HDFStore('../data/mystore.h5') 
model = LinearRegression()

byweek = store.select('df',column=['Producto_ID']).groupby('Semana')

byweek.apply(regress, 'Producto_ID', ['Venta_uni_hoy'])
#store.select('df',where =  "Producto_ID = 34053 & Semana = 3")

#byweek = store.select('df',"columns=['Producto_ID','Semana']").groupby(['Semana,Producto_ID'])
#byweek.count()
#byweek.apply(regress, 'Venta_uni_hoy', ['Producto_ID'])

#store.create_table_index('sales_prd', columns = ['Semana','Cliente_ID','Producto_ID','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima']\
                        #,optlevel=9, kind='full')

#for tempdf in pd.read_hdf('../data/mystore.h5', chunksize = 200000):
    #print(tempdf)
    #break
tempdf = pd.read_hdf('../data/mystore.h5', chunksize = 20000, where = "Producto_ID = 34053 or Producto_ID = 1146 \
    or Producto_ID = 41938 or Producto_ID = 2095 or Producto_ID = 8766 or Producto_ID = 30227")
df = pd.concat(tempdf)
df = df.rename(columns = {'Semana':'WeekNum','Venta_uni_hoy':'SaleUnits','Venta_hoy':'SaleNum',\
                'Dev_uni_proxima':'ReturnUnits','Dev_proxima':'ReturnNum', 'Demanda_uni_equil':'Demand',\
                'Agencia_ID':'SalesDepotId','Canal_ID':'SalesChannelId','Ruta_SAK':'RouteId','Cliente_ID':'ClientId',\
                'Producto_ID':'ProductId'})
#print(df.size)
#sb.barplot(data = df, x = ['ProductId'], y=['SaleUnits'])
#sb.pairplot(x_vars = ['WeekNum'], y_vars = ['SaleUnits','ReturnUnits'], data = df, size = 8)

#lm = smf.ols(formula='SaleUnits ~ WeekNum + ReturnUnits + SalesChannelId ', data=df).fit()
#lm.summary()
#df.groupby('WeekNum')['SaleUnits','ReturnUnits'].sum()

dfsubset = pd.DataFrame()
dfsubset['SaleUnits'] = df['SaleUnits']
dfsubset['WeekNum'] = df['WeekNum']
dfsubset['ReturnUnits'] = df['ReturnUnits']
#dfsubset['SalesChannelId'] = df['SalesChannelId']
#dfsubset['ProductId'] = df['ProductId']
#dfsubset['ClientId'] = df['ClientId']
#dfsubset['SalesDepotId'] = df['SalesDepotId']
saletrain, saletest, saleLabelTrain, saleLabelTest = train_test_split(dfsubset, dfsubset['ReturnUnits'], test_size = 0.4)    
%time print("%.16f" % np.sqrt(-cross_val_score(model, saletrain, saleLabelTrain, cv=10, scoring='mean_squared_error')).mean())
    #break
#1.05307379645e-12
    
# create a store

#for iterator in pd.read_csv('../data/train.csv', iterator = True, chunksize = 200000):
#    store.append('df', iterator, data_columns = ['Semana','Cliente_ID','Producto_ID',\
#                    'Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima'], index = False)
#store.create_table_index('df', 
#                         columns = ['Semana','Cliente_ID','Producto_ID','Venta_uni_hoy',\
#                                        'Venta_hoy','Dev_uni_proxima','Dev_proxima'],
#                         optlevel=9, 
#                      kind='full'
#                        )



    


OSError: ``../data`` does not exist

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
import numpy as np

model = LinearRegression()
tempdf = pd.read_hdf('../data/mystore.h5', chunksize = 20000, columns = "['Semana','Venta_uni_hoy','Dev_uni_proxima','Canal_ID']")
df = pd.concat(tempdf)
df = df.rename(columns = {'Semana':'WeekNum','Venta_uni_hoy':'SaleUnits',\
                'Dev_uni_proxima':'ReturnUnits','Canal_ID':'SalesChannelId'})
print(df.columns)
dftest = pd.read_csv('../data/test.csv', usecols=['id','Semana','Canal_ID'], header = 0)
dftest.rename(columns = {'Semana':'WeekNum','Canal_ID':'SalesChannelId'}, inplace=True)

dftest['SaleUnits'] = pd.Series()
dftest['ReturnUnits'] = pd.Series()

features = ['WeekNum','SalesChannelId']
X_train = df[features]
Y_train = df['SaleUnits']
model.fit(X_train, Y_train)
X_test = dftest[features]
Y_test = dftest['SaleUnits']
pred = model.predict(X_test)
pred